In [19]:
import pandas as pd
xls = pd.ExcelFile('Excel_test_dlouhy.xlsx')
sklad = pd.read_excel(xls, 'sklad_08_2018')
ceny = pd.read_excel(xls, 'ciselniky')
sklad_kopie=sklad.copy()
sklad_kopie

,ID_skladu,ID_komponenty,Mnozstvi,Stav_ke_dni
0,SK56,KO001,16,2018-08-01
1,SK00,KO037,47,2018-08-01
2,SK56,KO044,41,2018-08-01
3,SK97,KO160,12,2018-08-01
4,SK11,KO190,33,2018-08-01
...,...,...,...,...
116,SK02,ND107,6,2018-08-01
117,SK00,ND111,10,2018-08-01
118,SK02,ND114,32,2018-08-01
119,SK00,ND116,40,2018-08-01


# Sjednocení 2 tabulek cen do 1

In [20]:
ceny_komponentu=ceny.iloc[:,9:11]
jmena_cols=ceny_komponentu.iloc[0]
ceny_komponentu=ceny_komponentu[1:]
ceny_komponentu.columns=jmena_cols
ceny_komponentu=ceny_komponentu.rename(columns={'ID_komponenty':'ID_produktu'})
ceny_komponentu=ceny_komponentu.rename(columns={'Porizovaci_cena':'Prodejni_cena'})
ceny_komponentu

,ID_produktu,Prodejni_cena
1,KO001,11600
2,KO002,2550
3,KO003,13300
4,KO004,2350
5,KO005,14950
...,...,...
90,KO202,11250
91,KO203,12150
92,KO204,6400
93,KO205,8700


In [21]:
ceny_produktu=ceny.iloc[:,12:14]
jmena_cols=ceny_produktu.iloc[0]
ceny_produktu=ceny_produktu[1:]
ceny_produktu.columns=jmena_cols
ceny_produktu.reset_index()
ceny_produktu

,ID_produktu,Prodejni_cena
1,ND118,7350
2,ND112,11400
3,ND102,18975
4,ND113,20025
5,ND121,25050
...,...,...
90,NaN,NaN
91,NaN,NaN
92,NaN,NaN
93,NaN,NaN


In [22]:
ceny_celkem=pd.concat([ceny_produktu,ceny_komponentu],axis=0)
celkove=ceny_celkem.dropna()
celkove=celkove.reset_index(drop=True)
celkove

,ID_produktu,Prodejni_cena
0,ND118,7350
1,ND112,11400
2,ND102,18975
3,ND113,20025
4,ND121,25050
...,...,...
113,KO202,11250
114,KO203,12150
115,KO204,6400
116,KO205,8700


# Přiřazení ceny k věcem na skladech + celková cena (množství)

In [26]:
for index, komp in enumerate(sklad_kopie['ID_komponenty']):
    sklad_kopie['ceny'][index] = int(celkove.loc[celkove['ID_produktu']==komp]['Prodejni_cena'])
sklad_kopie

<ipython-input-26-03f9889aa39b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sklad_kopie['ceny'][index] = int(celkove.loc[celkove['ID_produktu']==komp]['Prodejni_cena'])


,ID_skladu,ID_komponenty,Mnozstvi,Stav_ke_dni,ceny
0,SK56,KO001,16,2018-08-01,11600
1,SK00,KO037,47,2018-08-01,14750
2,SK56,KO044,41,2018-08-01,12000
3,SK97,KO160,12,2018-08-01,11950
4,SK11,KO190,33,2018-08-01,5000
...,...,...,...,...,...
116,SK02,ND107,6,2018-08-01,98100
117,SK00,ND111,10,2018-08-01,119925
118,SK02,ND114,32,2018-08-01,87975
119,SK00,ND116,40,2018-08-01,60075


In [27]:
sklad_kopie['celk_cena']=sklad_kopie['ceny']*sklad_kopie['Mnozstvi']
sklad_kopie.groupby(['ID_skladu']).sum()['celk_cena']

ID_skladu
SK00     15779925
SK01      3206500
SK02     12651375
SK11      7510100
SK56     10008175
SK86A     8585175
SK97      8275350
Name: celk_cena, dtype: int64